In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


import sys
import os
sys.path.append(os.path.abspath(".."))  
sys.path.append(os.path.abspath("./src")) 



data_path = "../data/"
output_path = '../output/'

dataset ='Automatic_Labeling_Kyoogu/'
#dataset ='Dataset_1/'
#dataset ='Dataset_2/'

Full_DB_file = 'Labeled_Data_With_FCCs.xlsx'
Reduced_DB_file = 'Reduced_dataframe_FA2.xlsx'
isolated_observations_file = 'isolated_observations.xlsx'


df_full = pd.read_excel(data_path+dataset+Full_DB_file)
df_reduced = pd.read_excel(data_path+dataset+Reduced_DB_file,index_col=0)
df_isolated_observations = pd.read_excel(output_path+dataset+isolated_observations_file,index_col=0)

columns_to_keep =['File','Specie ID','Start','End','FminVoc','FmaxVoc']
df_filtered = df_full.loc[df_reduced.index, columns_to_keep]
df_filtered

,File,Specie ID,Start,End,FminVoc,FmaxVoc
0,SMA03126_20210611_200000.wav,BOAN,1.2,3.199,1.2,2500
1,SMA03126_20210611_200000.wav,BOAN,2.0,3.999,2.0,2500
2,SMA03126_20210611_200000.wav,BOAN,8.8,10.799,8.8,2500
3,SMA03126_20210611_200000.wav,BOAN,9.6,11.599,9.6,2500
4,SMA03126_20210611_200000.wav,BOAN,12.8,14.799,12.8,2500
...,...,...,...,...,...,...
1377,SMA03251_20210605_053000.wav,TRGL,54.0,55.999,54.0,7000
1378,SMA03251_20210608_073000.wav,TRGL,10.8,12.799,10.8,7000
1379,SMA03251_20210608_073000.wav,TRGL,32.8,34.799,32.8,7000
1380,SMA03251_20210608_073000.wav,TRGL,43.6,45.599,43.6,7000


In [9]:
# Rename the columns
columns_to_rename = {
    'Start': 'Begin Time (s)',
    'End': 'End Time (s)',
    'FminVoc': 'Low Freq (Hz)',
    'FmaxVoc': 'High Freq (Hz)'
}
df_filtered = df_filtered.rename(columns=columns_to_rename)

# Initialize the 'review' column in df_filtered with default values of 0
df_filtered['review'] = 0

# Set the 'review' column to 1 for rows where indices match those in df_isolated_observations
df_filtered.loc[df_filtered.index.isin(df_isolated_observations.index), 'review'] = 1
df_filtered

df_filtered.to_excel(output_path + dataset + "Labeled_Data_to_Review.xlsx", index=False)


In [10]:

# Create a dictionary to store the DataFrames
dataframes_dict = {}


# Iterate over the unique values in the 'File' column
for file_value in df_filtered['File'].unique():
    # Filter the original DataFrame for the current unique value
    df_filtered_copy = df_filtered[df_filtered['File'] == file_value].copy()
    
    # Add new columns
    df_filtered_copy['Selection'] = range(1, len(df_filtered_copy) + 1)
    df_filtered_copy['View'] = 'Spectrogram 1'
    df_filtered_copy['Channel'] = '1'

    # Reorder the columns
    columns_order = ['File', 'Selection', 'View', 'Channel'] + [col for col in df_filtered_copy.columns if col not in ['File', 'Selection', 'View', 'Channel']]
    df_filtered_copy = df_filtered_copy[columns_order]
    
    # Drop the 'File' column
    df_filtered_copy = df_filtered_copy.drop(columns=['File'])
    
    # Store the filtered DataFrame in the dictionary
    dataframes_dict[file_value] = df_filtered_copy
    
    # Save the DataFrame to a .txt file
    df_filtered_copy.to_csv(output_path + dataset + f"/SelectionTables/{file_value}.txt", sep='\t', index=False)

# Display the number of DataFrames created and the names of the keys in the dictionary
print(f"{len(dataframes_dict)} DataFrames have been created.")
print("The names of the keys in the dictionary are:", list(dataframes_dict.keys()))

# Display an example of the modified DataFrames
for key in list(dataframes_dict.keys())[:1]:  # Display only the first DataFrame as an example
    print(f"DataFrame for the file '{key}':")
    print(dataframes_dict[key].head())


123 DataFrames have been created.
The names of the keys in the dictionary are: ['SMA03126_20210611_200000.wav', 'SMA03247_20210605_200000.wav', 'SMA03330_20210603_191000.wav', 'SMA03328_20210606_200000.wav', 'SMA03251_20210605_194000.wav', 'SMA03393_20210609_200000.wav', 'SMA03294_20210603_193000.wav', 'SMA03406_20210601_191000.wav', 'SMA03247_20210606_200000.wav', 'SMA03286_20210604_200000.wav', 'SMA03247_20210606_191000.wav', 'SMA03328_20210606_195000.wav', 'SMA03393_20210602_190000.wav', 'SMA03326_20210611_183000.wav', 'SMA03406_20210608_194000.wav', 'SMA03251_20210604_200000.wav', 'SMA03330_20210609_192000.wav', 'SMA03406_20210601_200000.wav', 'SMA03393_20210602_193000.wav', 'SMA03286_20210608_200000.wav', 'SMA03320_20210328_054000.wav', 'SMA03175ICP_20210326_181000.wav', 'SMA03247_20210607_060000.wav', 'SMA03297_20210605_061000.wav', 'SMA03210_20210320_055000.wav', 'SMA03210_20210320_054000.wav', 'SMA03326_20210612_073000.wav', 'SMA03286_20210607_055000.wav', 'SMA03126_20210609_18